# Data Wrangling with MongoDB
## OpenStreetMap Project

The map area is the Triangle region of central North Carolina.  This area encompasses the three major cities of Raleigh, Durham and Chapel Hill and is home to technology hub Research Triangle Park.

I downloaded the data from [Mapzen](https://mapzen.com/data/metro-extracts) as a 465MB(uncompressed) OSM file.

![Triangle area - NC](Raleigh_map.png)

Map references: 
* http://gmaps-samples.googlecode.com/svn/trunk/fusiontables/rectangle_example.html

* http://gis.stackexchange.com/questions/29523/is-there-a-website-that-will-let-me-see-add-markers-and-bounding-boxes-on-top-of

___

In [2]:
import xml.etree.cElementTree as ET
import re
from collections import defaultdict, Counter

import codecs
import json
from pymongo import MongoClient

In [3]:
map_file = 'raleigh_north-carolina.osm'

### Definitions

* **Node** - A node represents a specific point on the earth's surface defined by its latitude and longitude.  Each node comprises at least an id number and a pair of coordinates.
<font color=orangeRed>Can I verify this in the dataset?</font>
* **Way** - A way defines <u>linear</u> features and area boundaries.  A way is an ordered list of between 2 and 2,000 nodes that define a polyline.  Ways are used to represent features such as rivers and roads.
* **Relations** - A relation is a multi-purpose data structure that documents a relationship between two or more data elements (nodes, ways and/or other relatations).  The relation's meaning is defined by its tags.  Typically the relation will have a 'type' tag. The relation's other tags need to be interpreted in light of the type tag.

All of the above can have one or more associated *tags* which describe the meaning of a particular element.

Reference: http://wiki.openstreetmap.org/wiki/Elements

### Create a dictionary of elements

SAX is Simple API for XML.  It's an event-driven mechanism for reading data from an XML document without having to read the entire DOM tree into memory. 

Goal: use iterative parsing to process the map file and figure out what and how many elements there are.  Create a dictionary with the element name as the key and the number of times this element is encountered as the value.


###### From Lesson 6 - iterative parsing

In [349]:
def count_tags(filename):
    dict_x = {}
    for _, elem in ET.iterparse(filename):
        #resource: http://stackoverflow.com/questions/473099/check-if-a-given-key-already-exists-in-a-dictionary-and-increment-it
        dict_x[elem.tag] = dict_x.get(elem.tag,0) + 1
    return dict_x
        

In [350]:
elements = count_tags(map_file)
elements

{'bounds': 1,
 'member': 7947,
 'nd': 2529113,
 'node': 2252660,
 'osm': 1,
 'relation': 797,
 'tag': 827245,
 'way': 224066}

##### From Lesson 6 - tag types

In [44]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        k = element.attrib['k']
        if (lower.search(k)):
            keys['lower'] += 1
        elif (lower_colon.search(k)):
            keys['lower_colon'] += 1
        elif (problemchars.search(k)):
            keys['problemchars'] += 1  
            print (k)
        else:
            keys['other'] += 1
    return keys

def process_keys(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

In [45]:
keys = process_keys(map_file)
keys

chair lift


{'lower': 509634, 'lower_colon': 273216, 'other': 44394, 'problemchars': 1}

*Note*: There is one key that contains problem characters: 'chair lift'
Apparently keys are not supposed to include spaces

##### From lesson 6 - exploring users

In [72]:
def get_user(element):
    return element.attrib['uid']

def process_users(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if 'uid' in element.attrib:
            users.add(get_user(element))
    return users

In [73]:
users = process_users(map_file)
len(users)

730

### Data auditing

##### Audit street type

In [173]:
#Note: \b means match at "word boundary", ie. match full words only
#      \S+ means match one or more of anything *except* a space character
#      \.? means match zero or more period characters
#      $ means perform this match from the end of the string
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons",
           "Circle","Extension","Fork","Highway","Terrace", "Way"]

# UPDATE THIS VARIABLE
street_type_mapping = { "St": "Street",
            "St.": "Street",
            "Rd." : "Road",
            "Ave" : "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "CIrcle": "Circle",
            "Cir": "Circle",
            "Ct" : "Court",
            "Dr" : "Drive",
           "Ext" : "Extension",
           "Ln" : "Lane",
           "Loop": "Loop",
           "Pkwy": "Parkway",
           "Pky": "Parkway",
           "Pl": "Place",
           "PI": "Place",
           "Rd": "Road",
           "Rd.": "Road",
           "ST": "Street",
           "St": "Street",
           "St,": "Street"
            }
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
def is_street_name(elem):
    return (elem.attrib['k'] == 'addr:street')

def audit_streets(osmfile):
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osmfile, events=('start',)):
        if elem.tag == 'node' or elem.tag == 'way':
            for tag in elem.iter('tag'):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    return street_types

def update_street_name(name, mapping):
    m = street_type_re.search(name)
    if m:
        #Use the replacement from the mapping if it exists.
        replacement = mapping.get(m.group(),0)
        if replacement:
            name = street_type_re.sub(replacement, name) 
    return name

##### Audit street direction

Want to identify streets that have an abbreviated directional component like: N, S, E, W at the beginning of the string.

In [298]:
street_direction = defaultdict(set)
street_direction_re = re.compile(r'^[S W|SW|S E|SE|N W|NW|N E|NE|N|E|W|S]\.?\s+', re.IGNORECASE)
# '(\s+[S W|SW|S E|SE|N W|NW|N E|NE|N|E|W|S]\.?$)|
#street_direction_re = re.compile(r'(\s+[West|S W|SW|S E|SE|N W|NW|N E|NE|N|E|W|S]\.?$)|(^[S W|SW|S E|SE|N W|NW|N E|NE|N|E|W|S]\.?\s+)')

for event, elem in ET.iterparse(map_file, events=('start',)):
        if elem.tag == 'node' or elem.tag == 'way':
            for tag in elem.iter('tag'):
                if is_street_name(tag):
                    street_name = tag.attrib['v']
                    m = street_direction_re.search(street_name)
                    if m:
                        direction_type = m.group()
                        street_direction[direction_type].add(street_name)


In [323]:
street_direction_mapping = {"E.": "East ",
                           "E": "East ",
                           "W": "West ",
                           "W.": "West ",
                           "S": "South ",
                           "S.": "South ",
                           "N": "North ",
                           "N.": "North "}


for value in street_direction.values():
    for street in value:
        print (street)
        m = street_direction_re.search(street)
        if m:
            sd = m.group()
            replacement = street_direction_mapping.get(sd.strip(),0)
            if replacement:
                street = street_direction_re.sub(replacement,street)
                print("   ==>",street)

E. Winmore Ave
   ==> East Winmore Ave
W Chapel Hill Street
   ==> West Chapel Hill Street
W EDENTON ST
   ==> West EDENTON ST
W Rosemary St.
   ==> West Rosemary St.
W Franklin St
   ==> West Franklin St
W. Franklin St.
   ==> West Franklin St.
W. Pettigrew St.
   ==> West Pettigrew St.
S Wilmington Street
   ==> South Wilmington Street
S Boylan Ave
   ==> South Boylan Ave
N Duke St Suite
   ==> North Duke St Suite
N Roxboro Rd
   ==> North Roxboro Rd
N Fordham Blvd.
   ==> North Fordham Blvd.
N Harrison Avenue
   ==> North Harrison Avenue
E Cabarrus Street
   ==> East Cabarrus Street
E Cameron Avenue
   ==> East Cameron Avenue
E Franklin Street
   ==> East Franklin Street
E Rosemary St.
   ==> East Rosemary St.
S. Boylan Ave
   ==> South Boylan Ave


##### From lesson 6 - improving street names

In [91]:
st_types = audit_streets(map_file)
for st_type, ways in st_types.items():
    for name in ways:
        better_name = update_street_name(name,street_type_mapping)
        if name != better_name:
            print (name,'=>', better_name)

defaultdict(set,
            {'100': {'100'},
             '1000': {'Six Forks Road #1000'},
             '17': {'US Highway 17'},
             '206': {'Barrett Dr Suite 206'},
             '501': {'US 15;US 501'},
             '54': {'Highway 54',
              'State Highway 54',
              'West Highway 54',
              'West NC Highway 54'},
             '55': {'Highway 55', 'NC Highway 55', 'US 55'},
             '70': {'US 70'},
             '751': {'NC Highway 751'},
             'Ave': {'Atlantic Ave',
              'E. Winmore Ave',
              'East Winmore Ave',
              'Fernway Ave',
              'Glenwood Ave',
              'Mountford Ave',
              'S Boylan Ave',
              'S. Boylan Ave'},
             'Blvd': {'Airport Blvd',
              'Capital Blvd',
              'Martin Luther King Jr Blvd',
              'Martin Luther King Jr. Blvd',
              'Martin Luther King Junior Blvd',
              'Southpoint Autopark Blvd',
              

##### Audit is_in

Was curious about this field, but after review am still not clear on what it's for.

In [154]:
is_in = {}
for event, elem in ET.iterparse(map_file, events=('start',)):
    if elem.tag == 'node' or elem.tag == 'way':
        for tag in elem.iter('tag'):
            if tag.attrib['k'] == 'is_in':
                value = tag.attrib['v']
                is_in[value] = is_in.get(value,0) + 1
is_in

{'Chatham,North Carolina,N.C.,NC,USA': 3,
 'Duke University': 47,
 'Durham,North Carolina,N.C.,NC,USA': 47,
 'Ellerbee Creek Watershed Association': 1,
 'North Carolina School of Science and Math': 13,
 'North Carolina,N.C.,NC,USA': 2,
 'Orange,North Carolina,N.C.,NC,USA': 79,
 'Pearsontown Elementary School': 1,
 'Raleigh, NC': 2,
 'Southwest Elementary School': 1,
 'US': 1,
 'Wake,North Carolina,N.C.,NC,USA': 26}

##### Audit locations

In [155]:
location = defaultdict(set)
for event, elem in ET.iterparse(map_file, events=('start',)):
    if elem.tag == 'node' or elem.tag == 'way':
        name = is_in = ''
        for tag in elem.iter('tag'):
            if tag.attrib['k'] == 'name':
                name = tag.attrib['v']
            if tag.attrib['k'] == 'is_in':
                is_in = tag.attrib['v']
        location[is_in].add(name)
location

defaultdict(set,
            {'': {'',
              'Stonecliff Drive',
              'Airport Drive',
              'Old Forge Circle',
              'East Martin Street',
              'Berryhill Drive',
              'Split Stone Lane',
              'Carpenter Fire Station Rd',
              'New Bern Ridge Drive',
              'Rambeau Circle',
              'Ridgehaven Townhomes',
              'Brooklyn Street',
              'Braddock Circle',
              'River Pine Dr',
              'Bashford Crest Lane',
              'Fleming Drive',
              'Rosewood Court',
              'Becton Residence Hall',
              'Cornell Street',
              'Wal-Mart: Store 1751',
              'Bedford Avenue',
              'Stokesbury Court',
              'Glendower Road',
              'South Knightsbridge Road',
              'Hogan Woods Circle',
              'Summerwood Townhomes',
              'Myers Park Drive',
              'Kilgore Avenue',
              'SAS Bui

##### Audit zip codes
Zip codes can be of the format 12345-1234.  Review the first part of the zip code.

In [127]:
def is_zip(elem):
    return (elem.attrib['k'] == 'addr:postcode')

def audit_zip(osmfile):
    zips = []
    for event, elem in ET.iterparse(osmfile, events=('start',)):
        if elem.tag == 'node' or elem.tag == 'way':
            for tag in elem.iter('tag'):
                if is_zip(tag):
                    zips.append(tag.attrib['v'].split('-')[0])
    return zips


In [129]:
zip_count = Counter(audit_zip(map_file))

In [156]:
zip_count.most_common()

[('27612', 1692),
 ('27560', 1515),
 ('27609', 1131),
 ('27519', 880),
 ('27701', 667),
 ('27705', 480),
 ('27615', 352),
 ('27510', 313),
 ('27514', 168),
 ('27513', 117),
 ('27511', 116),
 ('27606', 97),
 ('27707', 95),
 ('27601', 90),
 ('27516', 87),
 ('27517', 72),
 ('27704', 59),
 ('27703', 53),
 ('27713', 48),
 ('27617', 36),
 ('27603', 29),
 ('27613', 29),
 ('27607', 21),
 ('27604', 19),
 ('27610', 15),
 ('27605', 13),
 ('27614', 10),
 ('27608', 9),
 ('27695', 5),
 ('27162', 4),
 ('27518', 4),
 ('27616', 3),
 ('NC', 3),
 ('27895', 2),
 ('27599', 2),
 ('27602', 2),
 ('27708', 2),
 ('27709', 1),
 ('28616', 1),
 ('275198404', 1),
 ('2612', 1),
 ('275199', 1),
 ('275609194', 1),
 ('27502', 1),
 ('27710', 1),
 ('277030', 1)]

##### Audit cities

In [136]:
def is_city(elem):
    return (elem.attrib['k'] == 'addr:city')

def audit_city(osmfile):
    cities = []
    for event, elem in ET.iterparse(osmfile, events=('start',)):
        if elem.tag == 'node' or elem.tag == 'way':
            for tag in elem.iter('tag'):
                if is_city(tag):
                    cities.append(tag.attrib['v'])
    return cities

In [134]:
cities_count = Counter(audit_city(map_file))

In [135]:
cities_count

Counter({'Apex': 1,
         'Carrboro': 336,
         'Cary': 2668,
         'Chapel Hill': 470,
         'Chapel Hill, NC': 1,
         'Durham': 1287,
         'Morrisville': 1557,
         'Ra': 1,
         'Raleigh': 874,
         'Research Triangle Park': 1,
         'Wake Forest': 2,
         'cary': 1,
         'chapel Hill': 2,
         'durham': 2,
         'raleigh': 2})

##### Create a general audit function to return the values of a specified tag

In [142]:
def audit (osmfile, target_key):
    results = []
    for event, elem in ET.iterparse(osmfile, events=('start',)):
        if elem.tag == 'node' or elem.tag == 'way':
            for tag in elem.iter('tag'):
                if tag.attrib['k'] == target_key:
                    results.append(tag.attrib['v'])
    return results

In [143]:
cities_count = Counter(audit(map_file,'addr:city'))

In [157]:
house_number_count = Counter(audit(map_file,'addr:housenumber'))

In [165]:
house_numbers = audit(map_file,'addr:housenumber')

In [223]:
#What percentage of the house numbers are numeric:
numeric = 0
for i in house_numbers:
    if i.isnumeric():
        numeric += 1
print (numeric/len(house_numbers))

0.9815342934550121


In [169]:
first_digit = []
for i in house_numbers:
    first_digit.append(i[0])

In [171]:
Counter(first_digit)

Counter({'#': 3,
         '1': 4731,
         '2': 2741,
         '3': 1312,
         '4': 1708,
         '5': 1549,
         '6': 589,
         '7': 604,
         '8': 491,
         '9': 292,
         'A': 1,
         'B': 1,
         'J': 1,
         'S': 3})

In [164]:
house_number_count

Counter({'311 B': 1,
         '#200': 1,
         '5731': 1,
         '4135': 1,
         '238': 3,
         '5404': 9,
         '7033': 2,
         '1936': 1,
         '4800': 12,
         '4424': 1,
         '320': 15,
         '7108': 1,
         '8504': 8,
         '2722': 3,
         '1230': 2,
         'A': 1,
         '906': 4,
         '375': 1,
         '209': 61,
         '2009': 18,
         '4140': 1,
         '3560': 1,
         '401': 40,
         '1371': 1,
         '601 A': 1,
         '8313': 1,
         '3761': 1,
         '7328': 1,
         '3200': 4,
         '5779': 1,
         '4141': 1,
         '122': 23,
         '746': 2,
         '1401': 9,
         '805': 18,
         '5218': 1,
         '2429': 1,
         '3004 Suite C': 1,
         '130': 15,
         '5722': 1,
         '354': 1,
         '403': 42,
         '1257': 1,
         '9812': 1,
         '8414': 1,
         '7404': 1,
         '611': 10,
         '4015': 1,
         '812': 8,
         '1191': 

In [224]:
amenities = audit(map_file,'amenity')

In [227]:
Counter(amenities).most_common()

[('parking', 1884),
 ('place_of_worship', 535),
 ('bicycle_parking', 522),
 ('restaurant', 502),
 ('fast_food', 253),
 ('school', 221),
 ('fuel', 202),
 ('bench', 127),
 ('bank', 111),
 ('swimming_pool', 104),
 ('cafe', 94),
 ('waste_basket', 92),
 ('toilets', 85),
 ('atm', 76),
 ('pharmacy', 71),
 ('vending_machine', 65),
 ('bar', 57),
 ('pub', 47),
 ('fountain', 47),
 ('waste_disposal', 46),
 ('recycling', 46),
 ('post_box', 45),
 ('fire_station', 43),
 ('shelter', 43),
 ('dentist', 43),
 ('drinking_water', 39),
 ('library', 39),
 ('parking_entrance', 37),
 ('post_office', 35),
 ('doctors', 30),
 ('hospital', 27),
 ('college', 26),
 ('theatre', 26),
 ('car_wash', 24),
 ('grave_yard', 22),
 ('bbq', 18),
 ('kindergarten', 17),
 ('public_building', 16),
 ('clinic', 15),
 ('police', 14),
 ('cinema', 14),
 ('car_rental', 13),
 ('university', 13),
 ('veterinary', 12),
 ('childcare', 11),
 ('arts_centre', 11),
 ('community_centre', 9),
 ('bus_station', 9),
 ('marketplace', 7),
 ('courthouse

In [159]:
states = audit(map_file,'addr:state')

In [161]:
state_count = Counter(states)

In [162]:
state_count

Counter({'NC': 4267, 'NY': 1, 'North Carolina': 296, 'nc': 2})

___

### Data Cleaning

In [332]:
def clean_city(city):

    expected = ['Apex','Carrboro','Cary', 'Chapel Hill', 'Durham', 'Morrisville', 'Raleigh', 'Wake Forest']
    
    mapping = {
        'Chapel Hill, NC': 'Chapel Hill',
        'Ra': 'Raleigh',
        'Research Triangle Park': 'Durham'
    }
    if city not in expected:
        city = mapping.get(city, city) #if city is not in mapping, city = city.
    #Convert to title case               
    return city.title()
    

In [333]:
def clean_street(street):
    street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
    
    #These are street_type values that we'd expect
    expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons",
           "Circle","Extension","Fork","Highway","Terrace", "Way"]

    #Created this list through auditing of the current values in the data
    street_type_mapping = { "St": "Street",
            "St.": "Street",
            "Rd." : "Road",
            "Ave" : "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "CIrcle": "Circle",
            "Cir": "Circle",
            "Ct" : "Court",
            "Dr" : "Drive",
           "Ext" : "Extension",
           "Ln" : "Lane",
           "Loop": "Loop",
           "Pkwy": "Parkway",
           "Pky": "Parkway",
           "Pl": "Place",
           "PI": "Place",
           "Rd": "Road",
           "Rd.": "Road",
           "ST": "Street",
           "St": "Street",
           "St,": "Street"
            }
    
    m = street_type_re.search(street)
    
    if m:
        street_type = m.group()
        if street_type not in expected:
            #Use the replacement from the mapping if it exists.
            replacement = street_type_mapping.get(m.group(),0)
            if replacement:
                street = street_type_re.sub(replacement, street) 
                
    #Clean up street directions                   
    street_direction_re = re.compile(r'^[S W|SW|S E|SE|N W|NW|N E|NE|N|E|W|S]\.?\s+', re.IGNORECASE)

    street_direction_mapping = {"E.": "East ",
                           "E": "East ",
                           "W": "West ",
                           "W.": "West ",
                           "S": "South ",
                           "S.": "South ",
                           "N": "North ",
                           "N.": "North "}

    m = street_direction_re.search(street)

    if m:
        sd = m.group()
        replacement = street_direction_mapping.get(sd.strip(),0)
        if replacement:
            street = street_direction_re.sub(replacement,street)
                
    return street.title()                
    

In [334]:
def clean_zip(zip_code):    
    #Parse zip into components 
    #Compare against gold standard list from postal service
    #Flag zips that are too short or too long (length != 5)
    return zip_code

    

In [335]:
def clean_address(address_dict):
    #Fix street types
    if 'street' in address_dict:
        address_dict['street'] = clean_street(address_dict['street'])
    if 'city' in address_dict:
        address_dict['city'] = clean_city(address_dict['city'])
    if 'postcode' in address_dict:
        address_dict['postcode'] = clean_zip(address_dict['postcode'])
    #All address records should have a state, and the state should be NC
    address_dict['state'] = 'NC'        
    
    return address_dict

___

### Data Processing

##### From leson 6 - shape element

In [336]:
def shape_element(element):
    
    CREATED = [ "version", "changeset", "timestamp", "user", "uid"]
    POS = ['lon', 'lat']
    problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
    
    prob_list = []
    elem = {}
    created_dict = {}
    nd_list = []
    pos_array = []
    address_dict = {}
    
    #process only node and way tags
    if element.tag == 'node' or element.tag == 'way':
        #include the element type
        elem['type'] = element.tag
        for x in element.attrib:
            #attributes in the CREATED array should be added under a 
            #key called 'created'
            if x  in CREATED:
                created_dict[x] = element.attrib[x]
            #attributes that are not in CREATED or POS should be
            #turned into regular key/value pairs
            elif x not in POS:
                elem[x] = element.attrib[x]
            #attributes for longitude and lattitude should be 
            #added to a pos array
        try:
            pos_array = [float(element.attrib['lat']),
                float(element.attrib['lon'])]
        except KeyError:
            pass
        #For 'way' specifically, <nd ref... elements should be
        #turned into 'node_refs': [...]
        if element.tag == 'way':
            for nd in element.iter('nd'):
                nd_list.append(nd.attrib['ref'])
            if len(nd_list) > 0:
                elem['node_refs'] = nd_list
        for tag in element.iter('tag'):
            #if tag 'k' value contains problematic characters, ignore it
            if problemchars.search(tag.attrib['k']):
                break
            #work on address dictionary
            if tag.attrib['k'].startswith('addr:'):
                pieces = tag.attrib['k'].split(':')
                if pieces[1] == 'street' and len(pieces) != 2:
                    break
                #if tag value 'k' starts with 'addr:' and doesn't have
                #a second ':' after street, add it to address dictionary
                address_dict[':'.join(pieces[1:])] = tag.attrib['v'] 
            else:
                #title case all elements
                elem[tag.attrib['k']] = tag.attrib['v'].title()
        if len(created_dict) > 0:
            elem['created'] = created_dict
        if len(pos_array) > 0:
            elem['pos'] = pos_array
        if len(address_dict) > 0:
            elem['address'] = clean_address(address_dict)
        return elem
    else:
        return None

def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

In [337]:
data = process_map(map_file, False)

In [237]:
data[0]

{'_id': ObjectId('5679402901cc623e537407db'),
 'created': {'changeset': '7525913',
  'timestamp': '2011-03-11T18:03:13Z',
  'uid': '38487',
  'user': 'jumbanho',
  'version': '4'},
 'id': '21592692',
 'pos': [35.9746754, -78.9086172],
 'type': 'node'}

___

### Import to MongoDB
Also can do this on the command line using the `mongoimport` command

>mongoimport -db dbname -c collectionname --file input-file.json

In [4]:
client = MongoClient('localhost:27017')
db = client.OSM


In [234]:
db.Raleigh.find().count()

0

In [261]:
db.Raleigh.delete_many({})

In [262]:
db.Raleigh.insert_many(data)

In [263]:
db.Raleigh.find().count()

2476726

In [240]:
len(data)

2476726

In [241]:
db.Raleigh.find_one()

{'_id': ObjectId('5679402901cc623e537407db'),
 'created': {'changeset': '7525913',
  'timestamp': '2011-03-11T18:03:13Z',
  'uid': '38487',
  'user': 'jumbanho',
  'version': '4'},
 'id': '21592692',
 'pos': [35.9746754, -78.9086172],
 'type': 'node'}

In [242]:
db.Raleigh.find({"type":"node"}).count()

2252656

In [243]:
db.Raleigh.find({"type":"way"}).count()

224062

In [251]:
db.Raleigh.find({"type":"sewage"}).count()

3

In [276]:
len(db.Raleigh.distinct("created.user"))

716

In [5]:
db.Raleigh.aggregate([{"$match":{"type":{"$exists":1}}},])

In [11]:
pipeline = [{"$match":{"type":{"$exists":1}}}, {"$group":{"_id":"$type",
"count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":10}]

list(db.Raleigh.aggregate(pipeline))

[{'_id': 'node', 'count': 2252656},
 {'_id': 'way', 'count': 224062},
 {'_id': 'sewage', 'count': 3},
 {'_id': 'water', 'count': 1},
 {'_id': 'route', 'count': 1},
 {'_id': 'video', 'count': 1},
 {'_id': 'public', 'count': 1},
 {'_id': 'audio', 'count': 1}]

##### Get list of ammenities

In [10]:
pipeline = [{"$match":{"amenity":{"$exists":1}}}, {"$group":{"_id":"$amenity",
"count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":10}]

print (list(db.Raleigh.aggregate(pipeline)))

[{'_id': 'parking', 'count': 1925}, {'_id': 'place_of_worship', 'count': 544}, {'_id': 'bicycle_parking', 'count': 526}, {'_id': 'restaurant', 'count': 510}, {'_id': 'fast_food', 'count': 253}, {'_id': 'school', 'count': 226}, {'_id': 'fuel', 'count': 204}, {'_id': 'bench', 'count': 130}, {'_id': 'bank', 'count': 113}, {'_id': 'swimming_pool', 'count': 108}]


In [7]:
list(db.Raleigh.aggregate([{"$match":{"type":{"$exists":1}}}, {"$group":{"_id":"$type",
"count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":10}])
)


[{'_id': 'node', 'count': 2252656},
 {'_id': 'way', 'count': 224062},
 {'_id': 'sewage', 'count': 3},
 {'_id': 'water', 'count': 1},
 {'_id': 'route', 'count': 1},
 {'_id': 'video', 'count': 1},
 {'_id': 'public', 'count': 1},
 {'_id': 'audio', 'count': 1}]

___

### Create a sample of the osm file

Code is from instructor notes of of project submission instructions

Had to modify for python 3: wrapped the strings to be written to the OSM file in str.encode() in order to get the bytestring representation.

Also changed to pull every 50th top level element so that the file size would be small enough.  (Master file is about 450MB)

In [348]:
SAMPLE_FILE = "sample.osm"


def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write(str.encode('<?xml version="1.0" encoding="UTF-8"?>\n'))
    output.write(str.encode('<osm>\n  '))

    # Write every 50th top level element
    for i, element in enumerate(get_element(map_file)):
        if i % 50 == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write(str.encode('</osm>'))